In [1]:
# Importing Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
import re
import pandas as pd

In [2]:
executable_path = {'executable_path': './chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Scraping Mars News 
mars_news_url = "https://mars.nasa.gov/news/"
browser.visit(mars_news_url)

# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')




In [4]:
# Retrieve the latest element that contains news title and news_paragraph
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='rollover_description_inner').text

# Display scrapped data 
print(news_title)
print(news_p)


NASA Announces Landing Site for Mars 2020 Rover


After a five-year search, NASA has chosen Jezero Crater as the landing site for its upcoming Mars 2020 rover mission.



In [5]:
# Visit Mars Space Images through splinter module
image_url_featured = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url_featured)

In [6]:
# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Retrieve background-image url from style tag 
image_url = soup.find('article',class_='carousel_item')['style'].replace('background-image: url(','').replace(');', '')[1:-1]


# Website Url 
main_url = 'https://www.jpl.nasa.gov'
 
# Concatenate website url with scrapped route
featured_image_url = main_url + image_url

# Display full link to featured image
featured_image_url




'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17841-1920x1200.jpg'

In [7]:
# Scraping twitter page of Mars
url = 'https://twitter.com/marswxreport?lang=en'

# Browser visit
browser.visit(url)

# Create a Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
# Find all elements that contain tweets
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')
# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in latest_tweets: 
    mars_weather = tweet.find('p').text
    if 'InSight' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

InSight sol 158 (2019-05-07) low -99.7ºC (-147.5ºF) high -21.8ºC (-7.2ºF)
winds from the SSE at 4.8 m/s (10.7 mph) gusting to 13.6 m/s (30.4 mph)
pressure at 7.50 hPapic.twitter.com/8SrPjAhpGZ


In [9]:

# Visit Mars facts url 
facts_url = 'http://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
mars_facts = pd.read_html(facts_url)

# Find the mars facts DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = mars_facts[0]

# Assign the columns `['Description', 'Value']`
mars_df.columns = ['Description','Value']

# Set the index to the `Description` column without row indexing
mars_df = mars_df.set_index(['Description','Value'])



# Display mars_df
mars_df

,
Description,Value
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
# Scraping Mars Hemispheres
url_parent = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Browser visit
browser.visit(url_parent)

# Create a Beautiful Soup object
html = browser.html
soup = bs(html, 'html.parser')

# Child website links for each hemisphere
base_url = "https://astrogeology.usgs.gov"
links = [base_url + item.find(class_="description").a["href"] for item in soup.find_all("div", class_="item")]

In [11]:

# Create empty list for hemisphere urls 
hemisphere_image_urls = []


for url in links:
    
    # from url to soup
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Extract data
    title = soup.find("div", class_="content").find("h2", class_="title").text.replace(" Enhanced", "")
    img_url = base_url + soup.find("img", class_="wide-image")["src"]
    
    # Store in list
    hemisphere_image_urls.append({"title": title, "img_url": img_url})

# Display hemisphere_image_urls
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [12]:
browser.quit()